# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vila franca do campo,37.7167,-25.4333,16.23,85,100,3.39,PT,1735452834
1,1,aras-asan,8.8908,126.3033,27.21,87,69,2.91,PH,1735452835
2,2,tobelo,1.7284,128.0095,30.37,63,97,1.08,ID,1735452836
3,3,puerto ayora,-0.7393,-90.3518,23.08,97,52,2.80,EC,1735452837
4,4,adamstown,-25.0660,-130.1015,24.11,84,100,6.49,PN,1735452838


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"]).between (21, 27, inclusive = "both") &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
].dropna()

# # Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# # Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,mermoz boabab,14.7065,-17.4758,21.88,88,0,4.12,SN,1735452957
119,119,hermanus,-34.4187,19.2345,22.11,72,0,3.30,ZA,1735452970
226,226,sarupathar,26.1984,93.8094,24.28,40,0,2.02,IN,1735453093
307,307,san fernando de apure,7.8878,-67.4724,24.32,60,0,2.14,VE,1735453187
325,325,haiku-pauwela,20.9219,-156.3051,22.80,80,0,4.12,US,1735453207
326,326,kasane,-17.8167,25.1500,25.05,83,0,2.06,BW,1735453208
400,400,hilo,19.7297,-155.0900,21.16,91,0,0.89,US,1735453294
457,457,sittwe,20.1500,92.9000,26.54,39,0,3.54,MM,1735453079
507,507,salalah,17.0151,54.0924,26.05,36,0,3.09,OM,1735453426
515,515,sompeta,18.9333,84.6000,26.60,49,0,2.41,IN,1735453435


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample date
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
109,mermoz boabab,SN,14.7065,-17.4758,88,
119,hermanus,ZA,-34.4187,19.2345,72,
226,sarupathar,IN,26.1984,93.8094,40,
307,san fernando de apure,VE,7.8878,-67.4724,60,
325,haiku-pauwela,US,20.9219,-156.3051,80,
326,kasane,BW,-17.8167,25.1500,83,
400,hilo,US,19.7297,-155.0900,91,
457,sittwe,MM,20.1500,92.9000,39,
507,salalah,OM,17.0151,54.0924,36,
515,sompeta,IN,18.9333,84.6000,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit" : 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row ["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mermoz boabab - nearest hotel: La maison blanche
hermanus - nearest hotel: Aloe guest house
sarupathar - nearest hotel: No hotel found
san fernando de apure - nearest hotel: Hotel Plaza
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kasane - nearest hotel: Kasane Travel Lodge
hilo - nearest hotel: Dolphin Bay Hotel
sittwe - nearest hotel: Yuzana Aung Motel 1
salalah - nearest hotel: Muscat International Hotel
sompeta - nearest hotel: Haritha Beach Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
109,mermoz boabab,SN,14.7065,-17.4758,88,La maison blanche
119,hermanus,ZA,-34.4187,19.2345,72,Aloe guest house
226,sarupathar,IN,26.1984,93.8094,40,No hotel found
307,san fernando de apure,VE,7.8878,-67.4724,60,Hotel Plaza
325,haiku-pauwela,US,20.9219,-156.3051,80,Inn At Mama's Fish House
326,kasane,BW,-17.8167,25.1500,83,Kasane Travel Lodge
400,hilo,US,19.7297,-155.0900,91,Dolphin Bay Hotel
457,sittwe,MM,20.1500,92.9000,39,Yuzana Aung Motel 1
507,salalah,OM,17.0151,54.0924,36,Muscat International Hotel
515,sompeta,IN,18.9333,84.6000,49,Haritha Beach Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City",
    hover_cols = ["Humidity", "Hotel Name", "Country"] 
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)